<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=72248ccd0b5e70d2acf19f36c5a75d9b33306b0df48b1f6c0056d2881e101abc
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-08 14:23:34
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -56.72 K (-0.4%)
Current PnL: -21.07 L (-13.99%)
CY Booked + Current PnL: -9.63 L (-6.39%)
-------------------
Total profit:  1.43 L
Total loss:  -22.50 L
-------------------
Total Booked + Current PnL: 17.35 L (14.02%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.58 L (63.04%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.22,119361.0,18857.0,10253.0,3.75,18.76,8.59,28.96,79.0,1.84,0.85,59.10,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,39.0,M-SC,1.18,83283.0,-17494.0,17589.0,-0.10,-17.36,21.12,0.09,245.0,-0.99,0.60,9.42,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.94,49.0,H-MC,3.06,189660.0,24990.0,18340.0,-0.82,15.18,9.67,26.31,89.0,1.36,1.36,24.37,X40N,NTT,AC
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205541.0,1041.0,22260.0,-0.86,0.51,10.83,11.39,10.0,0.05,1.47,9.51,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205065.0,13323.0,24444.0,-0.37,6.95,11.92,19.69,99.0,0.55,1.47,14.22,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3813.51,59.0,M-SC,6.47,175763.0,-21369.0,111205.0,7.32,-10.84,63.27,45.57,236.0,-0.19,1.26,24.61,XY24,NTT,AUTO
37,INDIAMART,4810.62,-53.52,49.0,H-SC,9.15,129659.0,6323.0,125302.0,4.25,5.13,96.64,106.72,119.0,0.05,0.93,29.70,AR,ATH,MISC
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.22,119361.0,18857.0,10253.0,3.75,18.76,8.59,28.96,79.0,1.84,0.85,59.10,MH,ATH,METALS
40,INFY,2275.00,-18.16,53.0,H-LC,8.00,318307.0,5419.0,166284.0,2.46,1.73,52.24,54.87,3.0,0.03,2.27,8.50,X40,BTT,IT
48,LTIM,7230.20,-2.60,60.0,H-LC,2.46,241718.0,-4433.0,83634.0,1.95,-1.80,34.60,32.18,16.0,-0.05,1.73,33.59,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.0,-47.11,27.0,M-SC,37.78,52790.0,-12216.0,163142.0,-3.00,-18.79,309.04,232.18,198.0,-0.07,0.38,0.00,XY24,NTT,MISC
75,TITAGARH,1548.0,-4.07,51.0,H-SC,4.72,205086.0,-37091.0,147867.0,-2.99,-15.32,72.10,45.74,158.0,-0.25,1.47,33.39,XY24,NTT,ENGINEERING
55,RAJOOENG,143.1,-42.59,38.0,H-SC,15.99,87060.0,-15440.0,56041.0,-2.74,-15.06,64.37,39.61,114.0,-0.28,0.62,6.33,AR,ATH,MISC
82,VALIANTORG,838.0,-294.64,32.0,H-SC,1.71,108137.0,-55468.0,176782.0,-2.14,-33.90,163.48,74.15,139.0,-0.31,0.77,38.61,XR,NTT,CHEMICALS
23,DABUR,735.0,-13.56,26.0,H-MC,3.86,233770.0,-14298.0,122706.0,-2.08,-5.76,52.49,43.70,102.0,-0.12,1.67,7.85,XY24,BTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,43.54,49.0,M-MC,12.45,226030.0,1068.0,166584.0,-1.22,0.47,73.70,74.53,192.0,0.01,1.62,32.88,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-21.80,44.0,H-MC,1.91,134549.0,-1239.0,43998.0,-1.35,-0.91,32.70,31.48,107.0,-0.03,0.96,11.10,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.34,49.0,M-LC,5.86,151894.0,949.0,108513.0,0.60,0.63,71.44,72.51,53.0,0.01,1.09,6.69,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,60.0,H-SC,13.43,129195.0,-13284.0,32803.0,-1.18,-9.32,25.39,13.70,163.0,-0.40,0.92,52.49,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,39.0,M-SC,1.18,83283.0,-17494.0,17589.0,-0.10,-17.36,21.12,0.09,245.0,-0.99,0.60,9.42,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,2.00,220356.0,-49311.0,85652.0,-0.67,-18.29,38.87,13.47,138.0,-0.58,1.57,12.04,XY24,NTT,PAINTS
18,CERA,9475.0,-23.15,37.0,H-SC,1.59,140300.0,-35603.0,77628.0,0.18,-20.24,55.33,23.89,149.0,-0.46,1.00,21.17,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.87,50.0,H-MC,6.43,104253.0,-27582.0,69808.0,1.81,-20.92,66.96,32.03,98.0,-0.40,0.74,16.77,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,49.0,M-MC,12.45,226030.0,1068.0,166584.0,-1.22,0.47,73.70,74.53,192.0,0.01,1.62,32.88,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,6.91,214129.0,9053.0,103510.0,0.48,4.41,48.34,54.89,52.0,0.09,1.53,29.57,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.94,49.0,H-MC,3.06,189660.0,24990.0,18340.0,-0.82,15.18,9.67,26.31,89.0,1.36,1.36,24.37,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205065.0,13323.0,24444.0,-0.37,6.95,11.92,19.69,99.0,0.55,1.47,14.22,XY25,NTT,AC
17,CAMS,5211.76,-6.94,42.0,H-SC,1.74,105857.0,3853.0,40077.0,-0.53,3.78,37.86,43.06,122.0,0.10,0.76,21.54,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.22,119361.0,18857.0,10253.0,3.75,18.76,8.59,28.96,79.0,1.84,0.85,59.10,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.18,192681.0,12873.0,99539.0,-0.76,7.16,51.66,62.52,88.0,0.13,1.38,43.69,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.91,214129.0,9053.0,103510.0,0.48,4.41,48.34,54.89,52.0,0.09,1.53,29.57,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.52,49.0,H-SC,9.15,129659.0,6323.0,125302.0,4.25,5.13,96.64,106.72,119.0,0.05,0.93,29.70,AR,ATH,MISC
86,WIPRO,420.00,-13.34,49.0,M-LC,5.86,151894.0,949.0,108513.0,0.60,0.63,71.44,72.51,53.0,0.01,1.09,6.69,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.19,276573.0,-39269.0,151922.0,-1.48,-12.43,54.93,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-19.07,29.0,H-LC,5.70,204283.0,-47485.0,100691.0,-1.33,-18.86,49.29,21.13,27.0,-0.47,1.46,9.88,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205541.0,1041.0,22260.0,-0.86,0.51,10.83,11.39,10.0,0.05,1.47,9.51,X40N,ATH,PHARMA
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.67,40.0,H-LC,8.01,250950.0,-6113.0,41256.0,-0.31,-2.38,16.44,13.67,24.0,-0.15,1.79,17.55,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.39,54.0,H-LC,1.97,161395.0,-24700.0,72176.0,-0.70,-13.27,44.72,25.51,15.0,-0.34,1.15,18.94,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
48,LTIM,7230.2,-2.60,60.0,H-LC,2.46,241718.0,-4433.0,83634.0,1.95,-1.80,34.60,32.18,16.0,-0.05,1.73,33.59,X200,ATH,IT
51,NESTLEIND,1377.0,-10.51,50.0,H-LC,2.77,275558.0,10132.0,46652.0,0.07,3.82,16.93,21.40,11.0,0.22,1.97,10.71,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.79,44.0,H-LC,3.20,213642.0,3276.0,25509.0,-1.10,1.56,11.94,13.68,37.0,0.13,1.53,18.34,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.19,276573.0,-39269.0,151922.0,-1.48,-12.43,54.93,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
74,TCS,4406.34,-29.97,44.0,H-LC,12.18,279528.0,-66428.0,134677.0,-0.49,-19.20,48.18,19.73,1.0,-0.49,2.00,2.95,X40,ATH,IT
80,UNITDSPR,1644.00,-12.37,51.0,H-LC,6.77,229620.0,-4524.0,53157.0,-1.24,-1.93,23.15,20.77,86.0,-0.09,1.64,4.85,X40N,NTT,BREWERIES
1,ABB,7934.00,-40.94,41.0,H-LC,7.52,246672.0,-14947.0,134165.0,-1.54,-5.71,54.39,45.57,7.0,-0.11,1.76,4.96,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.39,54.0,H-LC,1.97,161395.0,-24700.0,72176.0,-0.70,-13.27,44.72,25.51,15.0,-0.34,1.15,18.94,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-19.07,29.0,H-LC,5.70,204283.0,-47485.0,100691.0,-1.33,-18.86,49.29,21.13,27.0,-0.47,1.46,9.88,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205541.0,1041.0,22260.0,-0.86,0.51,10.83,11.39,10.0,0.05,1.47,9.51,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.79,44.0,H-LC,3.20,213642.0,3276.0,25509.0,-1.10,1.56,11.94,13.68,37.0,0.13,1.53,18.34,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,44.0,H-LC,12.18,279528.0,-66428.0,134677.0,-0.49,-19.20,48.18,19.73,1.0,-0.49,2.00,2.95,X40,ATH,IT
40,INFY,2275.00,-18.16,53.0,H-LC,8.00,318307.0,5419.0,166284.0,2.46,1.73,52.24,54.87,3.0,0.03,2.27,8.50,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
83,VBL,671.64,-23.20,28.0,H-LC,10.19,276573.0,-39269.0,151922.0,-1.48,-12.43,54.93,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-40.94,41.0,H-LC,7.52,246672.0,-14947.0,134165.0,-1.54,-5.71,54.39,45.57,7.0,-0.11,1.76,4.96,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,30.76,78451.0,-35098.0,75148.0,-1.26,-30.91,95.79,35.27,268.0,-0.47,0.56,92.01,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,45.0,H-SC,10.88,91785.0,-6195.0,27802.0,-1.07,-6.32,30.29,22.05,152.0,-0.22,0.66,33.42,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.22,119361.0,18857.0,10253.0,3.75,18.76,8.59,28.96,79.0,1.84,0.85,59.10,MH,ATH,METALS
78,UJJIVANSFB,60.00,104.20,60.0,H-SC,13.43,129195.0,-13284.0,32803.0,-1.18,-9.32,25.39,13.70,163.0,-0.40,0.92,52.49,OX40N,NTT,BANKS
37,INDIAMART,4810.62,-53.52,49.0,H-SC,9.15,129659.0,6323.0,125302.0,4.25,5.13,96.64,106.72,119.0,0.05,0.93,29.70,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-182.95,64.0,M-SC,13.48,143901.0,-5319.0,149499.0,-1.96,-3.56,103.89,96.62,208.0,-0.04,1.03,55.73,XY25,NTT,FINANCE
69,SURYODAY,240.00,60.09,60.0,H-SC,7.34,147482.0,-31589.0,97323.0,-0.46,-17.64,65.99,36.71,135.0,-0.32,1.05,46.33,XR,NTT,BANKS
26,GREENPANEL,537.00,235.27,61.0,M-SC,5.09,149548.0,-33530.0,112505.0,0.15,-18.31,75.23,43.14,230.0,-0.30,1.07,40.67,XY24,NTT,MISC
32,HINDZINC,730.22,31.07,68.0,M-LC,6.91,214129.0,9053.0,103510.0,0.48,4.41,48.34,54.89,52.0,0.09,1.53,29.57,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.18,192681.0,12873.0,99539.0,-0.76,7.16,51.66,62.52,88.0,0.13,1.38,43.69,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.13
1,25,43.40
2,50,75.14


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.11
LC    32.17
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.07
X40      14.63
X40N     11.18
XR       10.93
XY25     10.38
AR        8.15
OX40N     7.68
X200      1.73
MH        1.61
X5K       1.53
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.98
H-LC    25.65
H-MC    20.50
M-SC    13.72
M-LC     5.51
M-MC     2.94
L-SC     1.41
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.72,-6.84,42.05
IT,12.79,-17.53,81.18
FINANCE,10.55,-14.35,69.57
MISC,6.92,-18.68,84.44
BANKS,6.30,-13.05,72.53
PAINTS,5.49,-20.09,37.99
ELECTRICAL,5.36,-9.62,48.08
INSURANCE,3.85,-2.76,41.17
AUTO,3.55,-15.99,71.44


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3230499.0,22
XR,1381347.0,14
AR,1188601.0,9
X40,791789.0,10
X40N,673421.0,9
OX40N,569128.0,10
XY25,485169.0,7
SR,263658.0,2
X5K,103510.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3734239.0,29
M-SC,1515606.0,17
H-MC,1285585.0,15
H-LC,1204321.0,15
M-LC,380192.0,4
M-MC,372138.0,2
L-SC,263312.0,3
L-MC,60235.0,1
L-LC,42627.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1198475.0      6
M-SC       XY24       844277.0      7
H-SC       AR         812084.0      5
           XR         807414.0      7
H-MC       XY24       577617.0      4
H-LC       X40        503767.0      5
M-MC       XY24       372138.0      2
H-SC       X40N       311490.0      4
           OX40N      263872.0      4
           SR         263658.0      2
H-LC       X40N       227339.0      3
H-MC       X40        214150.0      4
H-LC       AR         206341.0      2
H-MC       XY25       179626.0      2
L-SC       XR         174270.0      2
M-SC       AR         170176.0      2
M-LC       XY24       168169.0      2
M-SC       XY25       149499.0      1
           OX40N      146406.0      4
H-MC       X40N       134592.0      2
M-SC       XR         131376.0      2
H-LC       XY25       113417.0      3
M-LC       XR         108513.0      1
           X5K        103510.0      1
H-MC       XR          99539.0      1
L-SC       OX40N       89042.0      1
H-LC       X200        83634.0      1
H-SC       MH          77246.0      1
M-SC       X40         73872.0      1
H-LC       XY24        69823.0      1
H-MC       OX40N       69808.0      1
L-MC       XR          60235.0      1
L-LC       XY25        42627.0      1
H-MC       MH          10253.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
